In [1]:
import logging
import ast
from itertools import product
from large_gcs.graph_generators.contact_graph_generator import (
    ContactGraphGeneratorParams,
    ContactGraphGenerator,
)
from large_gcs.graph.incremental_contact_graph import IncrementalContactGraph
from large_gcs.graph.contact_graph import ContactGraph
from large_gcs.algorithms.search_algorithm import ReexploreLevel
from large_gcs.algorithms.gcs_astar import GcsAstar
from large_gcs.algorithms.gcs_astar_convex_restriction import GcsAstarConvexRestriction
from large_gcs.graph.contact_cost_constraint_factory import (
    contact_shortcut_edge_cost_factory_under,
    contact_shortcut_edge_cost_factory_over_obj_weighted,
)
from large_gcs.cost_estimators.shortcut_edge_ce import ShortcutEdgeCE
from large_gcs.cost_estimators.factored_collision_free_ce import FactoredCollisionFreeCE

from large_gcs.contact.contact_regions_set import ContactRegionParams

import numpy as np
from copy import deepcopy
from IPython.display import HTML


np.set_printoptions(formatter={"float": lambda x: "{0:0.3f}".format(x)})
logging.basicConfig(level=logging.INFO)
logging.getLogger("large_gcs").setLevel(logging.INFO)
logging.getLogger("drake").setLevel(logging.WARNING)

In [2]:
base_name = "cg_trichal2"

In [3]:
cg = ContactGraph.load_from_file(ContactGraphGeneratorParams.graph_file_path_from_name(base_name))

INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-01


100%|██████████| 3/3 [00:00<00:00, 61.19it/s]
INFO:large_gcs.graph.contact_graph:Loading 20 contact pair modes for 3 bodies...
INFO:large_gcs.graph.contact_graph:Generating contact sets for 179 sets...
100%|██████████| 179/179 [00:00<00:00, 448.86it/s]
INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 181/181 [00:00<00:00, 9052.92it/s]
INFO:large_gcs.graph.graph:Adding 181 vertices to graph...
100%|██████████| 181/181 [00:00<00:00, 10604.10it/s]
INFO:large_gcs.graph.contact_graph:Creating edge costs...
100%|██████████| 6120/6120 [00:00<00:00, 8086.01it/s]
INFO:large_gcs.graph.contact_graph:Creating edge constraints...
100%|██████████| 6120/6120 [00:01<00:00, 4434.67it/s]
INFO:large_gcs.graph.graph:Adding 6120 edges to graph...
100%|██████████| 6120/6120 [00:01<00:00, 4318.52it/s]
INFO:large_gcs.graph.contact_graph:Created contact graph: GraphParams(dim_bounds=(4, 18), n_vertices=181, n_edges=6120, source=[array([3.250, 0.000]), array([1.500, 0.500])], target=

In [4]:
cg_inc = IncrementalContactGraph.load_from_file(ContactGraphGeneratorParams.inc_graph_file_path_from_name(base_name))

INFO:large_gcs.graph.contact_graph:Creating vertex costs...
100%|██████████| 2/2 [00:00<00:00, 29959.31it/s]
INFO:large_gcs.graph.graph:Adding 2 vertices to graph...
100%|██████████| 2/2 [00:00<00:00, 11037.64it/s]
INFO:large_gcs.graph.contact_graph:Generating contact pair modes for 3 body pairs...
100%|██████████| 3/3 [00:00<00:00, 63.69it/s]
INFO:large_gcs.graph.incremental_contact_graph:Calculating adjacent contact pair modes (67)
100%|██████████| 67/67 [00:00<00:00, 4453.82it/s]


In [5]:
def are_all_graphs_edges_valid(g, verbose=False):
    no_intersections = []
    for (u,v) in g.edge_keys:
        if not g._check_intersection((g.vertices[u].convex_set.base_set, g.vertices[v].convex_set.base_set)):
            no_intersections.append((u,v))

    print(f"num invalid edges: {len(no_intersections)}/{len(g.edge_keys)}")
    if verbose:
        print(f"invalid edges: {np.array(no_intersections)}")

# are_all_graphs_edges_valid(cg)

In [8]:
are_all_graphs_edges_valid(cg_inc, verbose=True)

num invalid edges: 2/384
invalid edges: [["('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f3-rob0_v0')"
  'target']
 ["('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')"
  'target']]


In [ ]:
# First edge investigated
u = "('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f3-rob0_v0')"
v = "('NC|obs0_f2-obj0_f1', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f2-rob0_f2')"
v_contact_pair_mode_ids = ast.literal_eval(v)

# for mode_id in v_contact_pair_mode_ids:
#     print(f"cg_inc: {len(cg_inc._contact_pair_modes[mode_id].base_constraint_formulas)}, cg: {len(cg._contact_pair_modes[mode_id].base_constraint_formulas)}")

cg_inc._generate_vertex_neighbor(u, v_contact_pair_mode_ids)
v_cg = cg._create_contact_set_from_contact_pair_mode_ids(v_contact_pair_mode_ids)
v_cg_inc = cg_inc._create_contact_set_from_contact_pair_mode_ids(v_contact_pair_mode_ids)

print(cg._check_intersection((cg.vertices[u].convex_set.base_set, v_cg.base_set)))
print(cg._check_intersection((cg.vertices[u].convex_set.base_set, cg.vertices[v].convex_set.base_set)))
print((u, v) in cg.edges)

# # # This is different
# print(len(v_cg.base_constraint_formulas))
# print(len(v_cg_inc.base_constraint_formulas))

In [ ]:
print(len(cg._contact_pair_modes))
print(len(cg_inc._contact_pair_modes))
mode_id = 'NC|obs0_f0-obj0_v3'
# mode_id = 'NC|obj0_f2-rob0_f2'
print(len(cg._contact_pair_modes[mode_id].base_constraint_formulas))
print(len(cg_inc._contact_pair_modes[mode_id].base_constraint_formulas))

In [ ]:
cg.vertices[u].convex_set.base_set.ambient_dimension()

In [ ]:
np.allclose(cg_inc.vertices[u].convex_set.base_set.A(), cg.vertices[u].convex_set.base_set.A())

In [ ]:
def are_vertex_sets_the_same(u):
    return (np.allclose(cg_inc.vertices[u].convex_set.base_set.A(), cg.vertices[u].convex_set.base_set.A()) 
            and np.allclose(cg_inc.vertices[u].convex_set.base_set.A(), cg.vertices[u].convex_set.base_set.A()))

# are_vertex_sets_the_same("('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')")
# are_vertex_sets_the_same("('NC|obs0_f0-obj0_v3', 'NC|obs0_f1-rob0_f2', 'NC|obj0_f2-rob0_f2')")
are_vertex_sets_the_same("('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f0-rob0_v1')")

In [ ]:
def print_dim_A_b(vertex_name):
    print(f"\ncomparing dimensions of {vertex_name}")    
    print(f"shape of A cg: {cg.vertices[vertex_name].convex_set.base_set.A().shape}")
    print(f"shape of A cg_inc: {cg_inc.vertices[vertex_name].convex_set.base_set.A().shape}")
    print(f"shape of b cg: {cg.vertices[vertex_name].convex_set.base_set.b().shape}")
    print(f"shape of b cg_inc: {cg_inc.vertices[vertex_name].convex_set.base_set.b().shape}")

def print_dim_A_b_set(convex_set):
    print(f"\ncomparing base dimensions of {convex_set}")    
    print(f"shape of A: {convex_set.base_set.A().shape}")
    print(f"shape of b: {convex_set.base_set.b().shape}")

In [ ]:
for v in cg_inc.vertex_names:
    if v == "source" or v == "target": continue
    print_dim_A_b(v)

In [ ]:
print_dim_A_b(v)

In [ ]:
cg_inc.vertices[u].convex_set.base_set.A()

In [ ]:
cg_inc.vertices[u].convex_set.plot_base_set()

In [ ]:
(u, v) in cg_inc.edges

In [ ]:
cg_inc._modes_w_possible_edge_to_target

In [ ]:
cost_estimator_se = ShortcutEdgeCE(cg, contact_shortcut_edge_cost_factory_over_obj_weighted)
gcs_astar_se = GcsAstarConvexRestriction(
    cg,
    cost_estimator=cost_estimator_se,
    reexplore_level=ReexploreLevel.NONE,
)
sol_se = gcs_astar_se.run()

In [ ]:
cost_estimator = FactoredCollisionFreeCE(cg, use_combined_gcs=True, obj_multiplier=100)

gcs_astar = GcsAstarConvexRestriction(
    cg,
    cost_estimator=cost_estimator,
    reexplore_level=ReexploreLevel.NONE,
)
sol = gcs_astar.run()

In [ ]:
anim = cg.animate_solution()
HTML(anim.to_html5_video())

In [7]:
logging.getLogger("large_gcs.algorithms.gcs_astar_convex_restriction").setLevel(logging.DEBUG)
cost_estimator_se = ShortcutEdgeCE(cg_inc, contact_shortcut_edge_cost_factory_over_obj_weighted)
gcs_astar_se = GcsAstarConvexRestriction(
    cg_inc,
    cost_estimator=cost_estimator_se,
    reexplore_level=ReexploreLevel.NONE,
)
sol_se = gcs_astar_se.run()

INFO:large_gcs.algorithms.gcs_astar_convex_restriction:Running GcsAstarConvexRestriction, reexplore_level: ReexploreLevel.NONE
INFO:large_gcs.algorithms.gcs_astar_convex_restriction:visited_fd_vertices: {'source', 'target'}
INFO:large_gcs.algorithms.gcs_astar_convex_restriction:
n_vertices_visited: 2, n_edges_visited: 0, n_vertices_explored: 0, vertex_coverage: 0.667, edge_coverage: 0, time_wall_clock: 0, n_gcs_solves: 0, gcs_solve_time_total: 0, gcs_solve_time_iter_mean: 0, gcs_solve_time_last_10_mean: 0, gcs_solve_time_iter_std: 0, gcs_solve_time_iter_min: inf, gcs_solve_time_iter_max: 0, n_vertices_revisited: 0, n_vertices_reexplored: 0
now exploring node source's 1 neighbors (0)
DEBUG:large_gcs.algorithms.gcs_astar_convex_restriction:edge source -> ('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f3-rob0_v0') is feasible, new dist: 238.26001654659783, added to pq True
INFO:large_gcs.algorithms.gcs_astar_convex_restriction:visited_fd_vertices: {"('NC|obs0_f0-obj0_v3', 'NC|obs0_

In [ ]:
# First vertex investigated
# vertex_name = "('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f3-rob0_v0')"
# Second vertex investigated
vertex_name = "('NC|obs0_f0-obj0_v3', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')"
cg_edges = set([e.v for e in cg.outgoing_edges(vertex_name)])
cg_inc_edges = set([e.v for e in cg_inc.outgoing_edges(vertex_name)])
only_in_cg = np.array(list(cg_edges - cg_inc_edges))
only_in_cg_inc = np.array(list(cg_inc_edges - cg_edges))
print(f"{only_in_cg.size} Only in cg: \n{only_in_cg}")
print(f"{only_in_cg_inc.size} Only in cg_inc: \n{only_in_cg_inc}")

only_in_cg = only_in_cg[only_in_cg != "source"]

# mode_ids = ast.literal_eval(vertex_name)
# mode_ids_for_each_body_pair = []
# for id in mode_ids:
#     mode_ids_for_each_body_pair.append(
#         [id] + cg_inc._adj_modes[id]
#     )
# set_ids = list(product(*mode_ids_for_each_body_pair))
# set_ids_str = [str(set_id) for set_id in set_ids]
# in_cg_not_in_set_ids = np.array(list(set(only_in_cg) - set(set_ids_str)))
# print(f"{in_cg_not_in_set_ids.size} In cg but not in set_ids: \n{in_cg_not_in_set_ids}")

# for set_id in only_in_cg:
#     mode_ids = ast.literal_eval(set_id)
#     print(f"Generating {set_id}...")
#     cg_inc._generate_vertex_neighbor(vertex_name, mode_ids)

# # Verify that extra edges in cg actually have intersection
# for v in only_in_cg:
#     assert cg._check_intersection((cg.vertices[vertex_name].convex_set.base_set, cg.vertices[v].convex_set.base_set))

# # Verify that these sets are not empty
# for v in only_in_cg_inc:
#     assert not cg.vertices[v].convex_set.set.IsEmpty()

# # Are these not generated because they aren't in the adjacent modes?
# u_mode_ids = ast.literal_eval(vertex_name)
# for set_id in only_in_cg:
#     if set_id == "source": continue
#     v_mode_ids = ast.literal_eval(set_id)
#     print()
#     for i in range(len(u_mode_ids)):
#         print(f"{v_mode_ids[i]} in adj_modes to {u_mode_ids[i]}: {v_mode_ids[i] in cg_inc._adj_modes[u_mode_ids[i]] + [u_mode_ids[i]]}")


In [ ]:
cost_estimator = FactoredCollisionFreeCE(cg_inc, use_combined_gcs=True, obj_multiplier=100)

gcs_astar = GcsAstarConvexRestriction(
    cg_inc,
    cost_estimator=cost_estimator,
    reexplore_level=ReexploreLevel.NONE,
)
sol = gcs_astar.run()

In [ ]:
anim = cg.animate_solution()
HTML(anim.to_html5_video())

In [ ]:
# from itertools import product
# mode_ids_for_each_body_pair = [
#     ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7'],
#     ['B1', 'B2', 'B3',],
#     ['C1', 'C2', 'C3', 'C4', 'C5', 'C6'],
# ]
# mode_ids = ('A1', 'B1', 'C1')
# set_ids = list(product(*mode_ids_for_each_body_pair))
# print(f"Generating {len(set_ids)-1}")
# for set_id in set_ids:
#     if set_id != mode_ids:
#         print(f"Generating {set_id}")
#     else:
#         print(f"SKIPPING {set_id}")